<a href="https://colab.research.google.com/github/Prakhar-porwal/local-voice-tts-clean/blob/main/notebooks/fast_tts_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Local Voice TTS - Turbo Mode (Google Colab)

This notebook runs the **Local Voice TTS Backend** on a **Free Google Colab T4 GPU**.
This is roughly **10x-20x faster** than the Hugging Face CPU Free Tier.

### Instructions:
1.  **Runtime** -> **Change runtime type** -> **T4 GPU** (should be default).
2.  **Runtime** -> **Run all**.
3.  Wait for the **Public URL** (ending in `ngrok-free.app` or similar) to appear at the bottom.
4.  Copy that URL and use it in your Vercel Frontend!

In [8]:
# @title 1. Setup & Install (The "Binary Fix")
import os

# 1. Clean previous runs
if os.path.exists("local-voice-tts-clean"):
    !rm -rf local-voice-tts-clean

# 2. Clone Repository
!git clone https://github.com/Prakhar-porwal/local-voice-tts-clean
%cd local-voice-tts-clean

# 3. System Dependencies
!apt-get -y install espeak-ng libsndfile1

# 4. Install Dependencies (Transformers 4.40.0 is the KEY)
!pip install transformers==4.40.0
!pip install "numpy<2.0" setuptools wheel
# --- FIX: Reinstall Scipy to match Numpy ---
print("🔧 Reinstalling Scipy for Binary Compatibility...")
!pip install --force-reinstall scipy

# 5. Install Other Deps
!pip install accelerate fastapi "uvicorn[standard]" librosa soundfile pydub python-multipart pyngrok nest_asyncio

# --- CRITICAL MISSING PIECE ---
!pip install hatchling cython

# 6. Install TTS from Source (Community Fork)
print("Installing TTS... (This takes 60s)")
!pip install --no-build-isolation "git+https://github.com/idiap/coqui-ai-TTS.git#egg=coqui-tts"

print("\n✅ Setup Complete. NOW Restart Runtime & Run Cell 2.")

Cloning into 'local-voice-tts-clean'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 120 (delta 33), reused 118 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (120/120), 5.44 MiB | 16.14 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/local-voice-tts-clean/local-voice-tts-clean
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
🔧 Reinstalling Scipy for Binary Compatibility...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
coqui-tts 0.27.2 requires transformers>=4.57, but you have transformers 4.40.0 which is incompatible.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.8.0 which is incompatible.


Installing TTS... (This takes 60s)
  Cloning https://github.com/idiap/coqui-ai-TTS.git to /tmp/pip-install-rm3j8f_n/coqui-tts_fffddae8490b4418972c19b525e3d369
  Running command git clone --filter=blob:none --quiet https://github.com/idiap/coqui-ai-TTS.git /tmp/pip-install-rm3j8f_n/coqui-tts_fffddae8490b4418972c19b525e3d369
  Resolved https://github.com/idiap/coqui-ai-TTS.git to commit 268b532a510681803e0ac9fb2169ea95b4890566
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninst

ERROR: Operation cancelled by user
^C

✅ Setup Complete. NOW Restart Runtime & Run Cell 2.


In [ ]:
# @title 2. Start Turbo Backend (Bulletproof)
import os
import subprocess
import sys

# --- 1. FORCE CORRECT TRANSFORMERS VERSION ---
# The installer keeps upgrading it. We must downgrade it to work.
print("🔧 Enforcing Transformers 4.40.0...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==4.40.0"])

# --- 2. APPLY HOTFIX (The "isin" Bug) ---
target_file = "/usr/local/lib/python3.12/dist-packages/TTS/tts/layers/tortoise/autoregressive.py"
if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    if "isin_mps_friendly" in content:
        print("🔧 Patching TTS Code...")
        new_content = content.replace("from transformers.pytorch_utils import isin_mps_friendly as isin", "from torch import isin")
        with open(target_file, "w") as f:
            f.write(new_content)

# --- 3. START SERVER ---
print("🚀 Starting Server...")
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Auto-Accept License
os.environ["COQUI_TOS_AGREED"] = "1"

# Reset Ngrok
ngrok.kill()
ngrok.set_auth_token("36jQzzt9HifzkJJKUwtGTOAt3nA_3hZTGahebR2E5xk4WDX7o")
nest_asyncio.apply()

try:
    public_url = ngrok.connect(7860).public_url
    print("\n================================================================")
    print(f"🚀 TURBO API URL: {public_url}")
    print("================================================================\n")
except Exception as e:
    print(f"Ngrok Error: {e}")
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print(f"🚀 EXISTING URL: {tunnels[0].public_url}")

# Ensure we are in the right folder
if os.path.exists("/content/local-voice-tts-clean"):
    os.chdir("/content/local-voice-tts-clean")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = uvicorn.Config("main:app", host="127.0.0.1", port=7860)
server = uvicorn.Server(config)
import asyncio
await server.serve()

🔧 Enforcing Transformers 4.40.0...
🚀 Starting Server...

🚀 TURBO API URL: https://ocreate-shona-formatively.ngrok-free.dev

Using device: cpu
Loading VCTK model for preset voices...


100%|██████████| 148M/148M [00:03<00:00, 40.6MiB/s]
